<center><h1>LSTM for k-hot Encoded Multiclassification:</h1></center>

In [1]:
import numpy as np

from get_labels import get_labels
from sklearn.externals import joblib
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import ShuffleSplit

import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Activation, Reshape
from keras.layers import LSTM

Using TensorFlow backend.


<h4>Generate Random Training Lables:</h4>

In [2]:
"""
#Create the training labels
train_y = (np.random.randint(0,150,10000))
y = np.zeros((10000,150))
c = 0
for i in train_y: 
    y[c][i] = 1
    c = c+1
train_y = y.reshape(10000,1,150).reshape(10000,150)

#Random Multi encoded training labels
r = np.random.rand(10000,150)
r[r <= 0.3] = False
r[r > 0.3] = True
train_y[r == False] = 1
train_y = train_y.reshape(10000,1,150)
"""

'\n#Create the training labels\ntrain_y = (np.random.randint(0,150,10000))\ny = np.zeros((10000,150))\nc = 0\nfor i in train_y: \n    y[c][i] = 1\n    c = c+1\ntrain_y = y.reshape(10000,1,150).reshape(10000,150)\n\n#Random Multi encoded training labels\nr = np.random.rand(10000,150)\nr[r <= 0.3] = False\nr[r > 0.3] = True\ntrain_y[r == False] = 1\ntrain_y = train_y.reshape(10000,1,150)\n'

<h4> Load Training Labels: </h4>

In [3]:
labels = get_labels()

/home/ubuntu/.local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2881: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
labels_array = np.array([x for x in labels])
labels_reshaped = labels_array.reshape(1851243, 1, 1070)

<h4>Generate Training X Data:</h4>

In [8]:
#Create the training x data
#train_x = np.array([np.random.rand(1, 1000)[0] for i in range(1851243)]).reshape(1851243,1,1000)
train_x = joblib.load("/mnt/cleaned_tfidf_reduced_420_morning")

<h4> Load Samples: </h4>

In [10]:
print (train_x.shape)
train_x_reshaped = train_x.reshape(1851243,1,1000)
print (train_x_reshaped.shape)
#train_x = joblib.load("/mnt/saved_values/clean_tfidf_reduced")

(1851243, 1000)
(1851243, 1, 1000)


<h4> Create Train-Test Splits: </h4>

In [11]:
x_train, x_test, y_train, y_test = train_test_split(train_x_reshaped, labels_reshaped, test_size=0.20)

In [12]:
"""
print ("Raw Shapes:")
print("x_train_raw shape", x_train_raw.shape)
print("x_test_raw shape", x_test_raw.shape)

print("y_train_raw shape", y_train_raw.shape)
print("y_test_raw shape", y_test_raw.shape)

# split causes 1480994 training vs 370249 testing samples

x_train = x_train_raw.reshape(1480994, 1, 1000)
x_test = x_test_raw.reshape(370249, 1, 1000)

y_train = y_train_raw.reshape(1480994, 1, 1070)
y_test = y_test_raw.reshape(370249, 1, 1070)
"""

print ("\n New shapes:")
print("x_train shape", x_train.shape)
print("x_test shape", x_test.shape)

print("y_train shape", y_train.shape)
print("y_test shape", y_test.shape)


 New shapes:
('x_train shape', (1480994, 1, 1000))
('x_test shape', (370249, 1, 1000))
('y_train shape', (1480994, 1, 1070))
('y_test shape', (370249, 1, 1070))


<h4>Create A Sample Query to Predict On:</h4>

In [14]:
#Create the sample query 
sample = train_x[0].reshape(1,1,1000)

<h4>Create Multiclass Loss Function:</h4>

In [13]:
#Our custom loss function
def multiclass_loss(y_true, y_pred):
    EPS = 1e-5
    y_pred = K.clip(y_pred, EPS, 1 - EPS)
    return -K.mean((1 - y_true) * K.log(1 - y_pred) + y_true * K.log(y_pred))

<h4>Instantiate Parameters for LSTM:</h4>

In [14]:
#Our inputs for our model
shape = x_train.shape[2]
num_classes = 1070

<h4>Create and Compile LSTM:</h4>

In [22]:
#Function to create and compile our model
def create_model(shape,num_classes):
    print (type(shape), type(num_classes))
    print (shape, num_classes) # (None,shape)
    
    model = Sequential()
    model.add(LSTM(output_dim=32, input=(None, shape), return_sequences=True))
    model.add(Dense(num_classes, kernel_initializer='normal', activation='sigmoid'))
    model.compile(loss=multiclass_loss, optimizer='adam', metrics=['accuracy'])
    return model

<h4>Train LSTM</h4>

In [23]:
#Creating and training our model
model = create_model(shape, num_classes)
history = model.fit(train_x, train_y,
              batch_size=100, epochs=1,
              verbose = 1)

(<type 'int'>, <type 'int'>)
(1000, 1070)


TypeError: Expected int32, got <tensorflow.python.ops.variables.Variable object at 0x7f4e2699e990> of type 'Variable' instead.

<h4>Save LSTM Model</h4>

In [9]:
from keras.models import load_model   #save_model requires h5py, need to pip install h5py
model.save('khot_LSTM.h5') 

<h4>Load LSTM With Custom Loss Function:</h4>

In [20]:
#Loading our model with our custom loss function
model = load_model('khot_LSTM.h5', custom_objects={"multiclass_loss":multiclass_loss})

<h4>Obtain Prediction of Sample Query:</h4>

In [21]:
pred = model.predict(sample)[0][0]
#print(pred)
print('Our top prediction is: ',np.argmax(pred),"\n")
print("Indices of Top 3 predictions:")
top3 = pred.argsort()[-3:][::-1]
print(top3,"\n")
print("Indices where we have a 1:")
print(np.argwhere(train_y[0] == 1))
#print(pred)
#print(min(pred))
#print(max(pred))
#print(np.mean(pred))

Our top prediction is:  18 

Indices of Top 3 predictions:
[ 18  53 102] 

Indices where we have a 1:
[[  0   3]
 [  0   6]
 [  0  10]
 [  0  18]
 [  0  23]
 [  0  24]
 [  0  25]
 [  0  26]
 [  0  27]
 [  0  33]
 [  0  34]
 [  0  36]
 [  0  37]
 [  0  40]
 [  0  45]
 [  0  47]
 [  0  54]
 [  0  59]
 [  0  62]
 [  0  67]
 [  0  70]
 [  0  72]
 [  0  73]
 [  0  74]
 [  0  75]
 [  0  79]
 [  0  89]
 [  0  93]
 [  0  95]
 [  0 102]
 [  0 104]
 [  0 106]
 [  0 107]
 [  0 108]
 [  0 109]
 [  0 112]
 [  0 113]
 [  0 114]
 [  0 116]
 [  0 118]
 [  0 120]
 [  0 122]
 [  0 135]
 [  0 138]
 [  0 139]
 [  0 143]]
